# **Using LSTM to Predict traffic patterns in Manhattan**

As part of this cyberinfrastructure competition, teams must develop a machine learning model to help predict the movement of people in Manhattan. Due to the nature of the data, this might be confusing for some teams initially.

This notebook seeks to expand upon the Training dataset and show how the data can be used to generate predictions using a sample LSTM Model

Before running this notebook, please ensure that the TrainingData.h5 file provided in the competition introduction email by Professor Suining He

With that quick check completed, we first begin with importing some basic machine learning related libraries.

In [1]:
import h5py
import numpy as np
from datetime import datetime
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import LSTM, Dense, Reshape
from tensorflow.keras.optimizers import Adam

## Opening the Training Data

The TrainingData.h5 file is in HDF5 formatting. This isn't as common as .csv or .npy that many are familiar with. As such we'll first begin by just opening and reading the data.

The training dataset consists of 2 subdatasets, trip and timeslot

1) trip contains a list of bike traffic information of all the trips in Manhattan in a given time interval
2) timeslot contains a list of the corresponding time intervals of the trip data

Below we'll show how to open and access the trip and timeslot data

In [2]:
with h5py.File('TrainingData.h5', 'r') as f:
    # Access the trip dataset and their corresponding timestamps
    traffic_data = f['trip'][()]
    dates = f['timeslot'][()]

## Understanding the Data

Now let's examine the data and see what it looks like, as well as any potential preprocessing that we have to do

First off, the timeslot data, each timeslot corresponds to a 30 minute interval in the month of October 2019. Examining the first timeslot interval in the data we can see its taken in 2019-10-01 (October 1st 2019) at time 00:30 (from 12 am to 12:30 am). As such there are roughly 48x31 timeslots in total (1488 data samples total).


In [3]:
dates[0]

b'201910010030'

In [4]:
dates.shape

(1488,)

Next there's the trip data, each data sample consists of a 2x16x8 tensor. Here the first 16x8 matrix corresponds to how people moved out of different regions of Manhattan (outflow) and the second 16x8 matrix shows how people moved into different regions of Manhattan (inflow). With the 16x8 matrix being an abstracted map of Manhattan.


In [5]:
traffic_data[0]

array([[[ 0.,  0.,  0.,  0.,  1.,  0.,  0.,  1.],
        [ 0.,  0.,  3.,  4.,  4.,  0.,  1.,  0.],
        [ 0.,  1.,  0.,  8.,  0.,  1.,  0.,  0.],
        [ 0.,  0.,  2.,  5.,  1.,  2.,  1.,  0.],
        [ 0.,  2.,  1.,  1.,  1.,  1.,  0.,  0.],
        [ 0.,  0.,  2.,  1.,  5.,  2.,  0.,  0.],
        [ 0.,  2.,  1.,  1.,  0.,  1.,  0.,  0.],
        [ 0.,  4.,  4.,  6.,  7.,  0.,  0.,  0.],
        [ 1.,  1., 11.,  4.,  2.,  0.,  0.,  1.],
        [ 0.,  4.,  1.,  1.,  1.,  1.,  0.,  1.],
        [ 0.,  1.,  1.,  0.,  0.,  0.,  0.,  2.],
        [ 1.,  1.,  3.,  4.,  0.,  0.,  0.,  0.],
        [ 2.,  2.,  0.,  2.,  0.,  0.,  1.,  0.],
        [ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.]],

       [[ 0.,  0.,  0.,  0.,  1.,  0.,  0.,  1.],
        [ 0.,  0.,  0.,  1.,  0.,  1.,  1.,  0.],
        [ 0.,  1.,  1.,  3.,  0.,  1.,  2.,  0.],
        [ 0.,  1.,  0.,  0.,  1.,  3.,  0.,  0.]

In [6]:
traffic_data.shape

(1488, 2, 16, 8)

## Handling the Data

Before using the traffic and timeslot data. Some processing must be done to ensure its in a format our models can easily understand.

For the timeslot dates data, we'll just change them from the stored string format into a list where the year, month, day, hour, and minute are separated for ease of use

Teams are welcome to handle the data in any manner they choose when actually building and testing their models.

In [7]:
formatted_dates = []

for date_string in dates:
    formatted_date = datetime.strptime(date_string.decode(), '%Y%m%d%H%M')

    year = formatted_date.year
    month = formatted_date.month
    day = formatted_date.day
    hour = formatted_date.hour
    minute = formatted_date.minute

    formatted_dates.append(np.array([year, month, day, hour, minute]))

formatted_dates = np.array(formatted_dates).reshape(1488, 5, 1)

In [8]:
formatted_dates[0]

array([[2019],
       [  10],
       [   1],
       [   0],
       [  30]])

In [9]:
formatted_dates.shape

(1488, 5, 1)

Now with the data processed, we'll do a simple training and testing split given the data we have. For this tutorial we will use the last 5 days of October 2019 to test out model and the rest for training.

Note: As this data is time dependent, the data should not be shuffled during the train/test split process

In [10]:
test_size = 240

train_traffic_data = traffic_data[:-test_size]
test_traffic_data = traffic_data[-test_size:]

train_formatted_dates = formatted_dates[:-test_size]
test_formatted_dates = formatted_dates[-test_size:]

## Running the ML Models

First we'll begin by changing the names of our data to match standard conventions. It's important to note here the goal of the model is to take in some timeslot and predict the traffic data tensor at that timeslot.

As such the timeslot dates will be our X variable and the bike traffic data will be the target variable

In [11]:
X_train, X_test = train_formatted_dates, test_formatted_dates
y_train, y_test = train_traffic_data, test_traffic_data

Next, we'll define a basic LSTM model. Teams are welcome to choose whichever model architecture they are comfortable with.

In [12]:
# Basic LSTM Model
model = Sequential()
model.add(LSTM(50, activation='tanh', input_shape=(5, 1)))
model.add(Dense(2 * 16 * 8, activation='linear'))
model.add(Reshape((2, 16, 8)))

Next we'll compile and train the LSTM model. For this example, we'll be showing the Mean Square Error of the model at each epoch.

Thanks to our earlier work organizing the data, the process of actually training and testing our model is straightforward

In [13]:
model.compile(optimizer='adam', loss='mse')
model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/50
39/39 [==============================] - 1s 9ms/step - loss: 212.5657 - val_loss: 143.7969
Epoch 2/50
39/39 [==============================] - 0s 2ms/step - loss: 183.4570 - val_loss: 118.7862
Epoch 3/50
39/39 [==============================] - 0s 2ms/step - loss: 158.3965 - val_loss: 106.0134
Epoch 4/50
39/39 [==============================] - 0s 2ms/step - loss: 143.9914 - val_loss: 98.1775
Epoch 5/50
39/39 [==============================] - 0s 2ms/step - loss: 134.2470 - val_loss: 93.1884
Epoch 6/50
39/39 [==============================] - 0s 2ms/step - loss: 127.3423 - val_loss: 89.9704
Epoch 7/50
39/39 [==============================] - 0s 2ms/step - loss: 122.3828 - val_loss: 87.9088
Epoch 8/50
39/39 [==============================] - 0s 2ms/step - loss: 118.8026 - val_loss: 86.6221
Epoch 9/50
39/39 [==============================] - 0s 2ms/step - loss: 116.1038 - val_loss: 85.7970
Epoch 10/50
39/39 [==============================] - 0s 2ms/step - loss: 114.2159 - val_

When examining the model's results during the training stage it's important to make sure the model's loss gets lower as the number of epochs grows. This is a good indication that the model is actually learning some of the underlying trends in the data.

In [14]:
mse = model.evaluate(X_test, y_test)

# Show rmse to see how model performs on the test set
rmse = np.sqrt(mse)
print(f'Root Mean Squared Error: {rmse}')

8/8 [==============================] - 0s 967us/step - loss: 62.4032
Root Mean Squared Error: 7.899569497269297


This basic model can definitely capture some trends present in the data, but it's clear from the rmse score and loss seen above that there are some things that this model isn't seeing.

Also this example will not delve into things like hyperparameter tuning, but this is a good way to try and get some extra performance out of your model. Just make sure to not overfit your hyperparameters to the data as these models will be tested later on with a separate dataset as part of the final judging criteria.

Next we'll cover how to save and reload your trained models. This is a very important step as your models will be tested later on as well so having your models saved will make this process much easier. This is especially true for larger models that may take a long time to train. Below we show a quick example of how to save and reload the weights of your model

In [15]:
model.save('lstm_model.keras')

In [16]:
new_model = load_model('lstm_model.keras')

mse = new_model.evaluate(X_test, y_test)

# Show rmse to see how model performs on the test set
rmse = np.sqrt(mse)
print(f'Root Mean Squared Error: {rmse}')

8/8 [==============================] - 0s 1ms/step - loss: 62.4032
Root Mean Squared Error: 7.899569497269297


In [17]:
from keras.models import Sequential
from keras.layers import LSTM, Dense, Reshape
from keras.callbacks import Callback

class PrintYPredCallback(Callback):
    def on_epoch_end(self, epoch, logs=None):
        y_pred = self.model.predict(X_test)
        print(f"Predictions for Epoch {epoch + 1}:")
        print(y_pred)

# Define your model
model = Sequential()
model.add(LSTM(50, activation='tanh', input_shape=(5, 1)))
model.add(Dense(2 * 16 * 8, activation='linear'))
model.add(Reshape((2, 16, 8)))
model.compile(optimizer='adam', loss='mse')

# Fit the model with the custom callback
model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test), callbacks=[PrintYPredCallback()])


Epoch 1/50
8/8 [==============================] - 1s 1ms/step loss: 221.690
Predictions for Epoch 1:
[[[[-1.88099537e-02 -3.46565433e-02  1.54355395e+00 ...  4.77071494e-01
     6.06406212e-01  1.37867010e+00]
   [-6.66583627e-02  1.17861843e+00  9.69879806e-01 ...  1.36463296e+00
     1.60491860e+00  1.23475850e+00]
   [-3.49647552e-02  1.56907320e+00  6.35080695e-01 ...  8.31073940e-01
     1.41633499e+00  8.60131204e-01]
   ...
   [ 1.40348625e+00  1.51018655e+00 -1.81400683e-03 ...  5.29774278e-02
    -7.16406107e-02  4.46225971e-01]
   [ 1.30767119e+00 -3.37661952e-02  1.09316611e+00 ...  7.15267301e-01
    -1.48811743e-01  8.47820416e-02]
   [-2.05981806e-02 -1.89495236e-02  1.08373404e+00 ...  8.78032804e-01
     9.41216588e-01  2.18617007e-01]]

  [[ 1.20933853e-01  4.29482758e-03  6.46305978e-01 ...  1.49678016e+00
     9.03927028e-01  1.52851105e-01]
   [-1.68518215e-01  9.06732142e-01  1.07274711e+00 ...  8.74265671e-01
     9.66543436e-01  1.36764872e+00]
   [ 6.35585096e-0

8/8 [==============================] - 0s 869us/steposs: 185.88
Predictions for Epoch 2:
[[[[ 1.90449487e-02 -2.33155154e-02  2.71987915e+00 ...  2.75675797e+00
     2.54199982e+00  4.14680815e+00]
   [ 2.51447782e-02  3.98020768e+00  4.35796404e+00 ...  4.83064795e+00
     4.60688686e+00  2.30962086e+00]
   [-2.25417875e-03  4.25186253e+00  2.98912549e+00 ...  3.83319926e+00
     4.61343002e+00  6.37424171e-01]
   ...
   [ 4.69736099e+00  5.22291279e+00 -1.23036373e-02 ...  7.04048574e-02
    -6.90678880e-03  4.74904567e-01]
   [ 4.34587717e+00  1.11043872e-02  2.39918423e+00 ...  4.92927253e-01
    -3.38619053e-02  1.08680157e-02]
   [-9.49919038e-03  3.07437219e-02  1.95144916e+00 ...  2.05388522e+00
     9.99455273e-01  2.01051459e-01]]

  [[-9.12929047e-03  2.96776742e-02  2.67538500e+00 ...  3.62749648e+00
     2.40398574e+00  3.26098394e+00]
   [ 7.23549537e-03  3.98381996e+00  4.21710539e+00 ...  3.56669760e+00
     3.68931675e+00  2.51937556e+00]
   [-4.74689528e-03  3.6251182

8/8 [==============================] - 0s 847us/steposs: 158.60
Predictions for Epoch 3:
[[[[-2.56327074e-02  5.15888445e-04  2.55594563e+00 ...  2.64291382e+00
     2.54899693e+00  5.38374329e+00]
   [ 2.89668478e-02  5.19595003e+00  6.44258165e+00 ...  6.92537069e+00
     4.61885548e+00  2.14730620e+00]
   [-1.48212910e-02  4.72401714e+00  4.41878700e+00 ...  5.71797609e+00
     6.72219515e+00  6.86272442e-01]
   ...
   [ 6.58477926e+00  7.34577847e+00 -2.81328149e-02 ...  4.95758653e-02
     1.29622687e-02  4.50362533e-01]
   [ 5.22199392e+00 -1.01788361e-02  2.07611442e+00 ...  4.72299159e-01
    -8.96491669e-03 -3.51443514e-03]
   [ 2.80547328e-02 -2.26924066e-02  1.78214943e+00 ...  1.95472801e+00
     1.09623706e+00  8.36742595e-02]]

  [[-3.94317228e-03 -7.99118076e-04  2.30772662e+00 ...  3.29347086e+00
     2.14803743e+00  4.85073614e+00]
   [ 2.97821313e-02  5.27814531e+00  6.36971569e+00 ...  5.80102777e+00
     4.17474842e+00  2.38587022e+00]
   [-1.09042013e-02  4.6384267

8/8 [==============================] - 0s 821us/steposs: 143.63
Predictions for Epoch 4:
[[[[-4.94378991e-03  3.49557959e-05  2.54262161e+00 ...  2.66582036e+00
     2.53118968e+00  5.64256239e+00]
   [ 2.06168108e-02  5.53242540e+00  7.77958059e+00 ...  8.22012615e+00
     4.52093220e+00  2.18621826e+00]
   [-9.18813050e-04  4.66347313e+00  4.99853086e+00 ...  6.50430012e+00
     7.85449219e+00  7.51337349e-01]
   ...
   [ 7.68008041e+00  8.54375839e+00  1.07667642e-03 ...  2.62575410e-02
     5.33191673e-03  4.60274935e-01]
   [ 5.26035690e+00 -3.93631496e-03  2.16189218e+00 ...  6.09538853e-01
    -7.30438903e-03 -1.41536258e-03]
   [ 1.81793291e-02 -1.50444619e-02  1.89994276e+00 ...  1.91841054e+00
     1.16663551e+00  1.97762474e-01]]

  [[-5.42888418e-04  4.55593690e-05  2.34150934e+00 ...  3.25826859e+00
     2.24011517e+00  5.44438839e+00]
   [ 2.39097700e-02  5.53322172e+00  7.64315891e+00 ...  7.25289488e+00
     4.14748192e+00  2.40103269e+00]
   [-1.16165355e-03  4.7532959

8/8 [==============================] - 0s 813us/steposs: 133.25
Predictions for Epoch 5:
[[[[ 1.00991689e-04 -1.07959379e-04  2.58820033e+00 ...  2.66559219e+00
     2.51052308e+00  5.69042206e+00]
   [ 1.07461307e-02  5.69668865e+00  8.73261356e+00 ...  9.16534519e+00
     4.53233051e+00  2.13323998e+00]
   [-5.30898571e-04  4.68055725e+00  5.14499092e+00 ...  6.94311142e+00
     8.59209824e+00  6.77202046e-01]
   ...
   [ 8.33324432e+00  9.34105587e+00 -6.84043276e-04 ...  7.36844093e-02
     1.71088055e-03  3.63153845e-01]
   [ 5.31537914e+00 -1.40438136e-03  2.17998433e+00 ...  6.09534979e-01
    -2.94538215e-03  2.92584300e-05]
   [ 8.81316885e-03 -7.32458010e-03  1.75672603e+00 ...  1.93725395e+00
     1.00761509e+00  1.11054011e-01]]

  [[ 1.67378224e-04  4.00378369e-04  2.34088087e+00 ...  3.26686025e+00
     2.18407869e+00  5.67236090e+00]
   [ 1.46651790e-02  5.66027355e+00  8.53754425e+00 ...  8.32718563e+00
     4.11945438e+00  2.32601404e+00]
   [ 4.99064103e-04  4.7760100

...

8/8 [==============================] - 0s 1ms/step loss: 73.14
Predictions for Epoch 46:
[[[[-2.74514332e-02  2.24160939e-03  2.84423858e-01 ...  1.17921852e-01
     9.16618779e-02  1.37297666e+00]
   [-2.69139111e-02  6.98794127e-01  2.39840627e+00 ...  2.45019007e+00
     5.17989397e-01  4.92448628e-01]
   [ 3.39679979e-02  5.52425325e-01  1.37102056e+00 ...  1.54989839e+00
     2.15060091e+00  2.09306806e-01]
   ...
   [ 1.67583025e+00  2.35165691e+00 -1.21195968e-02 ... -2.70851180e-02
    -2.25500260e-02  1.68366376e-02]
   [ 5.77728927e-01 -1.49483830e-02 -1.27753289e-02 ... -7.22929761e-02
     1.36923799e-02 -5.66579588e-03]
   [-5.30213118e-03  2.09589675e-03  1.63241297e-01 ...  3.09105903e-01
    -1.16068926e-02 -2.52532847e-02]]

  [[ 1.14275012e-02  4.53645084e-03  2.25799039e-01 ...  2.48345822e-01
     3.85202914e-01  1.03652072e+00]
   [ 6.01371564e-03  9.43567336e-01  2.20324373e+00 ...  2.58589840e+00
     6.84743583e-01  3.95769894e-01]
   [-2.16325056e-02  9.9347132

8/8 [==============================] - 0s 964us/steposs: 71.90
Predictions for Epoch 47:
[[[[-3.01061273e-02 -2.32043816e-03  2.71381736e-01 ...  6.72689974e-02
     1.02067329e-01  1.34047735e+00]
   [-2.47924905e-02  6.23711824e-01  2.35711241e+00 ...  2.35430861e+00
     5.04324198e-01  4.83983964e-01]
   [ 3.30873691e-02  4.97759879e-01  1.37848735e+00 ...  1.50091290e+00
     2.09396291e+00  1.98126331e-01]
   ...
   [ 1.59218347e+00  2.27235556e+00 -1.26807354e-02 ... -1.23632550e-02
    -1.81208868e-02  2.93970704e-02]
   [ 5.11119843e-01 -1.47220194e-02 -4.90995944e-02 ... -5.66743463e-02
     1.63499527e-02 -4.54463437e-03]
   [-5.57847694e-03  6.23209495e-03  1.75124139e-01 ...  2.79307187e-01
    -5.40755428e-02 -7.29606487e-03]]

  [[ 1.14404382e-02  6.41250424e-03  2.13010356e-01 ...  1.81834906e-01
     3.50338042e-01  1.03591859e+00]
   [ 3.86808068e-04  9.00316775e-01  2.12908053e+00 ...  2.55470014e+00
     6.94292724e-01  3.88573945e-01]
   [-1.82789303e-02  9.5636022

8/8 [==============================] - 0s 1ms/step loss: 73.092
Predictions for Epoch 48:
[[[[-3.76322009e-02 -2.84375064e-03  2.51610488e-01 ...  6.68668598e-02
     4.86533754e-02  1.19764984e+00]
   [-3.35545763e-02  5.08590162e-01  2.24542904e+00 ...  2.16773796e+00
     4.21474606e-01  4.45420504e-01]
   [ 3.92223187e-02  4.56050098e-01  1.39263296e+00 ...  1.40830338e+00
     1.97143507e+00  1.77153364e-01]
   ...
   [ 1.46710587e+00  2.12165117e+00 -1.46529060e-02 ... -4.09135967e-02
    -2.47878954e-02 -2.75554024e-02]
   [ 4.16768193e-01 -1.82905942e-02 -8.52696747e-02 ... -1.20541885e-01
     1.56414006e-02 -7.09390081e-03]
   [-4.75694239e-03  2.76224455e-03  1.68883920e-01 ...  2.70729750e-01
    -3.40454243e-02 -1.49643188e-02]]

  [[ 1.35861933e-02  8.15087836e-03  2.26257548e-01 ...  1.07956968e-01
     3.23758692e-01  9.96588886e-01]
   [-2.05267221e-04  8.38273287e-01  2.02644825e+00 ...  2.39750528e+00
     5.99805236e-01  3.33897263e-01]
   [-2.26769876e-02  9.028336

8/8 [==============================] - 0s 1ms/step loss: 69.39
Predictions for Epoch 49:
[[[[-3.06856800e-02 -4.33690846e-04  2.65036851e-01 ...  5.96951619e-02
     9.16814357e-02  1.25743079e+00]
   [-2.52938941e-02  5.29869258e-01  2.31247616e+00 ...  2.23108006e+00
     4.74421948e-01  5.03471076e-01]
   [ 3.49962004e-02  4.73298252e-01  1.41455317e+00 ...  1.43624306e+00
     2.02487993e+00  2.02598199e-01]
   ...
   [ 1.50786912e+00  2.18769026e+00 -1.28896181e-02 ... -2.67196111e-02
    -2.25032009e-02  1.12021603e-02]
   [ 4.59446996e-01 -1.66844968e-02 -3.18714865e-02 ... -7.76700154e-02
     1.26018962e-02 -9.05397162e-03]
   [-5.82955033e-03  3.88934044e-03  1.96548745e-01 ...  2.93799579e-01
    -2.40847692e-02 -1.68700516e-02]]

  [[ 8.42259359e-03  6.44215057e-03  2.28156209e-01 ...  1.35459229e-01
     3.43600869e-01  1.05403733e+00]
   [-4.73685563e-04  8.47093165e-01  2.07960296e+00 ...  2.48200154e+00
     6.87593579e-01  3.92627656e-01]
   [-2.18762942e-02  9.4336402

8/8 [==============================] - 0s 990us/steposs: 69.94
Predictions for Epoch 50:
[[[[-3.17755267e-02 -7.24415295e-04  2.49967009e-01 ...  3.34323570e-02
     8.58893842e-02  1.20475912e+00]
   [-2.64329072e-02  4.84825045e-01  2.28729343e+00 ...  2.19058561e+00
     4.48568165e-01  4.88819718e-01]
   [ 3.35588343e-02  4.43487793e-01  1.43574309e+00 ...  1.41707063e+00
     2.00223494e+00  1.74913615e-01]
   ...
   [ 1.45316219e+00  2.14628792e+00 -1.30733699e-02 ... -1.55462716e-02
    -2.16723811e-02  2.97155827e-02]
   [ 4.20833945e-01 -1.64739620e-02 -6.49092346e-02 ... -9.06899199e-02
     1.17552094e-02 -6.81998022e-03]
   [-3.59760597e-03  3.38031352e-03  1.75011754e-01 ...  3.06251764e-01
    -5.50523102e-02 -5.11108898e-03]]

  [[ 8.44266452e-03  7.03838095e-03  2.52348006e-01 ...  8.05241615e-02
     3.28560323e-01  1.07198095e+00]
   [-9.91072506e-04  8.38082373e-01  2.06642318e+00 ...  2.43703461e+00
     6.49297774e-01  4.05753255e-01]
   [-1.98646449e-02  9.2807877


model2=Sequential()
model2.add((LSTM(20,activation='tanh',input_shape=(5, 1)))
model2.add(Dense(2*16*8,activation='sigmoid'))
model2.add(Reshape((2, 16, 8)))


This concludes this tutorial notebook. Please keep in mind that you don't need to replicate this tutorial in your final models. Teams are welcome to change their model's architectures, change their model's hyperparameters, etc. this is a time to learn and explore different topics within the scope of machine learning. Also you might want to think of performing normalization of the data as well as K-fold cross validation to improve the model learning.

If any teams have follow up questions or need any help relating to this hackathon please feel free to reach out via email or on the Microsoft Teams Q&A Chat. We are all more than happy to help answer any questions you might have!

# **Thank you and best of luck in the competition!**